In [1]:
import numpy as np
import pandas as pd
import logging
import os
import glob
import regex as re
import torch
import argparse
import random
import json
import itertools
import ast
import sys
import ast
from tqdm import tqdm
import warnings
import pickle as pk
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import pipeline

warnings.filterwarnings("ignore")

/home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("conll_modified_prompt.txt", "r") as text_file:
    prompt = text_file.read()
print (prompt)

Entity Types: [Peop, Loc, Org]
Relation Types: [Work_For, Live_In, Located_In, OrgBased_In, Kill]

TEXT: Penry raped Pamela Moseley Carpenter on Oct. 15 , 1979 , in Livingston , Texas , then stabbed her to death .
Relation List: [["Livingston", "Located_In", "Texas"]]</s>

TEXT: For singer and guitarist Bob Boyd , 57 , the Ranch has been home since he left Chattanooga , Tenn. , 35 years ago .
Relation List: [["Bob Boyd", "Live_In", "Chattanooga"],["Bob Boyd", "Live_In", "Tenn."],["Chattanooga", "Located_In", "Tenn."]]</s>

TEXT: Rome is in Lazio province and Naples in Campania .
Relation List: [["Rome", "Located_In", "Lazio"],["Naples", "Located_In", "Campania"]]</s>

TEXT: He has been the State Department 's refugee coordinator and is a former Harvard politics professor. _Ronald for the rank of ambassador during his tenure as U.S. negotiator on textile matters.
Relation List: [["State Department", "OrgBased_In", 'U.S."],["Harvard", "OrgBased_In", "U.S."]]</s>

TEXT: In another scene ,

In [3]:
relation_types = ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"]

In [4]:
conll = json.load(open('conll04_dev.json'))

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl", 
                                              cache_dir="/scratch/wadhwa.s/cache", 
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl", 
                                          cache_dir="/scratch/wadhwa.s/cache")

In [6]:
generator = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer)

In [64]:
prompt = """In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] and provide reasonable explanation.
TEXT: Rome is in Lazio province and Naples in Campania .
RELATIONS: [["Rome", "Located_In", "Lazio"],["Naples", "Located_In", "Campania"]]
Explanation: Rome is located in Lazio, and naples is located in Campania.


In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] and provide reasonable explanation.
TEXT: David Foster is the AP 's Northwest regional reporter , based in Seattle.
RELATIONS: [['David Foster", "Work_For", "AP"],["AP", "OrgBased_In", "Northwest"],["AP", "OrgBased_In", "Seattle"]]
Explanation: Dave Foster works for AP, which is an organization based in the Northwest Seattle region.


In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] and provide reasonable explanation.
TEXT: SAN FRANCISCO ( AP )
RELATIONS: [["AP", "OrgBased_In", "SAN FRANCISCO"]]
Explanation: AP is an organization based in San Francisco.


In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] and provide reasonable explanation.
TEXT: He graduated from high school from Benton , Tenn. and from Tennessee Tech in Cookville , and holds a doctorate in physics from Virginia Tech ."""

In [65]:
print (prompt)
torch.cuda.empty_cache()

In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] and provide reasonable explanation.
TEXT: Rome is in Lazio province and Naples in Campania .
RELATIONS: [["Rome", "Located_In", "Lazio"],["Naples", "Located_In", "Campania"]]
Explanation: Rome is located in Lazio, and naples is located in Campania.


In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] and provide reasonable explanation.
TEXT: David Foster is the AP 's Northwest regional reporter , based in Seattle.
RELATIONS: [['David Foster", "Work_For", "AP"],["AP", "OrgBased_In", "Northwest"],["AP", "OrgBased_In", "Seattle"]]
Explanation: Dave Foster works for AP, which is an organization based in the Northwest Seattle region.


In the given text, find all relation triplets of these relation-types only: ["Work_For", "Live_In", "OrgBased_In", "Located_In", "Kill"] a

In [66]:
res = generator(prompt, max_length=200)

In [67]:
print (res[0]["generated_text"])

RELATIONS: [["Tennessee Tech", "Located_In", "Cookville"], ["Tennessee Tech", "Courses_Offered", "Physics"], ["Tennessee Tech", "Current_President", "James E. Benton"], ["Tennessee Tech", "Current_President", "James E. Benton"], ["Tennessee Tech", "Current_President", "James E. Benton"], ["Tennessee Tech", "Current_President", "James E. Benton"], ["Tennessee Tech", "Current_President", "James E. Benton"], ["Tennessee Tech", "Current_


In [10]:
force_words_ids = [
#     tokenizer([force_word], add_prefix_space=True, add_special_tokens=False).input_ids,
    tokenizer(relation_types, add_special_tokens=False).input_ids,
]

In [11]:
dev_conll = []
for i in tqdm(conll[:2]):

    p =  prompt + "TEXT: " + ' '.join(i["tokens"]) + "\nRelation List: "
    dev_conll.append(p)
#     print (p)
#     print ("\n--------------\n")

100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 41943.04it/s]


In [8]:
torch.cuda.empty_cache()

In [15]:
ip = []
response = []

for ix, instance in enumerate(tqdm(dev_conll)):
        res = generator(instance, 
              max_length=100,
              force_words_ids=force_words_ids,
              num_beams=10, 
              num_return_sequences=2,
              early_stopping=True)
        torch.cuda.empty_cache()
        ip.append(instance.split("\n")[-2])
        print (instance.split("\n")[-2])
        print (res[0]["generated_text"])
        print ("\n--------\n")
        response.append(res[0]["generated_text"])

 50%|█████████████████████████████████████████▌                                         | 1/2 [07:46<07:46, 466.71s/it]

TEXT: John Wilkes Booth , who assassinated President Lincoln , was an actor .
[["John Wilkes Booth", "Occupation", "Actor"]] nowiki>["John Wilkes Booth", "Assassination of Abraham Lincoln"], ["John Wilkes Booth", "Assassination of Abraham Lincoln"], ["John Wilkes Booth", "Assassination of Abraham Lincoln"], ["John Wilkes Booth", "Assassination of Lincoln Kill

--------



100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [15:44<00:00, 472.34s/it]

TEXT: The opera company performed at the Palace of Fine Arts , in San Francisco , on June 30 and July 1-2 , said Kevin O 'Brien , a spokesman for the theater.
[["Palace of Fine Arts", "Location", "San Francisco"], ["Kevin O'Brien", "Speaker for the theater"]] nowiki>["Palace of Fine Arts", "Location", "Palace of Fine Arts, San Francisco", "June 30 and July 1-2"]] nowiki>["Palace of Fine Arts", Kill

--------



In [34]:
ip = []
response = []

for ix, instance in enumerate(tqdm(dev_conll)):
    try:
        res = generator(instance, 
              max_length=200)
        torch.cuda.empty_cache()
        ip.append(instance.split("\n")[-2])
        print (instance.split("\n")[-2])
        print (res[0]["generated_text"])
        print ("\n--------\n")
        response.append(res[0]["generated_text"])
    except:
        torch.cuda.empty_cache()
        pass

 50%|██████████████████████▌                      | 1/2 [00:08<00:08,  8.07s/it]

TEXT: John Wilkes Booth , who assassinated President Lincoln , was an actor .
[["John Wilkes Booth", "Occupation", "Actor"]]

--------



100%|█████████████████████████████████████████████| 2/2 [00:17<00:00,  8.80s/it]

TEXT: The opera company performed at the Palace of Fine Arts , in San Francisco , on June 30 and July 1-2 , said Kevin O 'Brien , a spokesman for the theater.
[["Kevin O'Brien", "Worked_For", Palace of Fine Arts]]

--------



In [ ]:
df = pd.DataFrame({'input': ip, 'response': response}, index=None)
#df.to_csv("conll_flan_fewshot_results.csv", index=False)